In [ ]:
import pandas as pd
import numpy as np

# 1. Load CSV
csv_path = "../WebScraping/Transactions.csv"
df = pd.read_csv(csv_path)

# 2. Drop duplicate Txn Hashes
df = df.drop_duplicates(subset=["Txn Hash"])

# 3. Standardize & rename columns
df = df.rename(columns=lambda c: (
    c.strip()
     .lower()
     .replace(" ", "_")
     .replace("(", "")
     .replace(")", "")
     .replace("-", "_")
))

df = df.rename(columns={
    "valuecint": "value_cint",    # rename the raw
    "txn_fee":   "txn_fee_cint",
})
# now drop any leftover
if "valuecint" in df.columns:
    df = df.drop(columns=["valuecint"])
# e.g. "Txn Hash" → "txn_hash", "Txn Fee (CINT)" → "txn_fee_cint"

# 4. Parse and normalize timestamps
df["timestamp_clean"] = (
    df["timestamp"]
      .astype(str)
      .str.split("\n")
      .str[-1]                # take the 'Jul, 15, 2025, 03:03' portion
)

df["timestamp"] = pd.to_datetime(df["timestamp_clean"], format="%b, %d, %Y, %H:%M", errors="coerce")
df = df.drop(columns="timestamp_clean")

# 5. Normalize address fields
for col in ["from", "to", "contract_address"]:
    if col in df:
        df[col] = df[col].str.lower().str.strip().replace("", np.nan)

# 6. Convert all on-chain numeric columns to floats/ints
df["block"]        = df["block"].astype(int)
df["value_cint"]   = df["value_cint"].astype(float)
df["txn_fee_cint"] = df["txn_fee_cint"].astype(float)


# 7. Derive domain-specific fields

# 7b. Classify txn direction for a given “our” address set
#    (you can supply your own key-addresses list)
our_addrs = {"0xabc…", "0xdef…"}  
def direction(row):
    if row["from"] in our_addrs and row["to"] not in our_addrs:
        return "outgoing"
    elif row["to"] in our_addrs and row["from"] not in our_addrs:
        return "incoming"
    else:
        return "other"
df["direction"] = df.apply(direction, axis=1)

# 8. Drop or archive any truly irrelevant fields
#    e.g., if ‘status’ is always ‘Success’, drop it; if you’ll never need raw input_data, drop that.
if "status" in df and df["status"].nunique() == 1:
    df = df.drop(columns=["status"])
for col in ["input_data", "logs"] :
    if col in df:
        df = df.drop(columns=[col])

# 9. Sort & re-index
df = df.sort_values("timestamp").reset_index(drop=True)

# 10. Final integrity checks
assert df["txn_hash"].is_unique, "Txn Hash is not unique!"
assert df[["block", "timestamp"]].notnull().all().all(), "Missing block or timestamp!"



df = df.rename(columns={
    "from": "from_address",
    "to":   "to_address"
})

df.head(10)

#output_path = "Transactions_cleaned.csv"
#df.to_csv(output_path, index=False)


In [1]:
import pandas as pd
import numpy as np

# Prepare a list to hold (step_name, row_count)
row_counts = []

# 1. Load CSV
csv_path = "../WebScraping/Transactions.csv"
df = pd.read_csv(csv_path)
row_counts.append(("1. after load", df.shape[0]))

# 2. Drop duplicate Txn Hashes
df = df.drop_duplicates(subset=["Txn Hash"])
row_counts.append(("2. after drop_duplicates", df.shape[0]))

# 3. Standardize & rename columns
orig_cols = df.columns.tolist()
df = df.rename(columns=lambda c: (
    c.strip()
     .lower()
     .replace(" ", "_")
     .replace("(", "")
     .replace(")", "")
     .replace("-", "_")
))
df = df.rename(columns={
    "valuecint": "value_cint",
    "txn_fee":   "txn_fee_cint",
})
if "valuecint" in df.columns:
    df = df.drop(columns=["valuecint"])
row_counts.append(("3. after rename columns", df.shape[0]))

# 4. Parse and normalize timestamps
df["timestamp_clean"] = (
    df["timestamp"]
      .astype(str)
      .str.split("\n")
      .str[-1]
)
df["timestamp"] = pd.to_datetime(
    df["timestamp_clean"],
    format="%b, %d, %Y, %H:%M",
    errors="coerce"
)
df = df.drop(columns="timestamp_clean")
row_counts.append(("4. after parse timestamps", df.shape[0]))

# 5. Normalize address fields
for col in ["from", "to", "contract_address"]:
    if col in df:
        df[col] = (
            df[col]
              .str.lower()
              .str.strip()
              .replace("", np.nan)
        )
row_counts.append(("5. after normalize addresses", df.shape[0]))

# 6. Convert numeric types
df["block"]        = df["block"].astype(int)
df["value_cint"]   = df["value_cint"].astype(float)
df["txn_fee_cint"] = df["txn_fee_cint"].astype(float)
row_counts.append(("6. after convert numeric", df.shape[0]))

# 7. Derive direction
our_addrs = {"0xabc…", "0xdef…"}
def direction(row):
    if row["from"] in our_addrs and row["to"] not in our_addrs:
        return "outgoing"
    elif row["to"] in our_addrs and row["from"] not in our_addrs:
        return "incoming"
    else:
        return "other"
df["direction"] = df.apply(direction, axis=1)
row_counts.append(("7. after direction", df.shape[0]))

# 8. Drop irrelevant fields
if "status" in df and df["status"].nunique() == 1:
    df = df.drop(columns=["status"])
for col in ["input_data", "logs"]:
    if col in df:
        df = df.drop(columns=[col])
row_counts.append(("8. after drop irrelevants", df.shape[0]))

# 9. Sort & re-index
df = df.sort_values("timestamp").reset_index(drop=True)
row_counts.append(("9. after sort & reset_index", df.shape[0]))

# 10. Final rename
df = df.rename(columns={
    "from": "from_address",
    "to":   "to_address"
})
row_counts.append(("10. after final rename", df.shape[0]))

# Build a summary table
summary = pd.DataFrame(row_counts, columns=["step", "n_rows"])
summary["dropped"] = summary["n_rows"].shift(1) - summary["n_rows"]
summary = summary.fillna(0).astype({"dropped": int})

print(summary)



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\datta\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\datta\AppData\Roaming\Python\Python311\site-packages\traitlets\config\application.py", line 1043, in launch_instance
    app.start()
  File "C:\Users\datta\AppData\Roaming\Python\Python311\site-packages\ipykernel\kernelapp.py", line 736, in start
    self.io_lo

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\datta\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\datta\AppData\Roaming\Python\Python311\site-packages\traitlets\config\application.py", line 1043, in launch_instance
    app.start()
  File "C:\Users\datta\AppData\Roaming\Python\Python311\site-packages\ipykernel\kernelapp.py", line 736, in start
    self.io_lo

AttributeError: _ARRAY_API not found

                           step  n_rows  dropped
0                 1. after load    5180        0
1      2. after drop_duplicates    1030     4150
2       3. after rename columns    1030        0
3     4. after parse timestamps    1030        0
4  5. after normalize addresses    1030        0
5      6. after convert numeric    1030        0
6            7. after direction    1030        0
7     8. after drop irrelevants    1030        0
8   9. after sort & reset_index    1030        0
9        10. after final rename    1030        0


In [2]:
unique_hashes = df["txn_hash"].unique()

# number of unique hashes
print(f"Found {len(unique_hashes)} unique transaction hashes")

# (optional) view them
for h in unique_hashes:
    print(h)

Found 1030 unique transaction hashes
0x10927...b039478
0x09cf8...b2bace6
0xdae48...30a14d6
0x3e78c...fbce879
0xbcbdf...537d2e2
0xe670c...c90f019
0x4caa0...01d215f
0xae111...046aef8
0x933d8...3fe2293
0x12fb5...2b558a7
0xe48a8...74b5eae
0xd988a...d3b685d
0x821eb...3a55b93
0x54616...00d73dc
0x91af3...b71def6
0x47a61...bf4c9d5
0x34579...24437d3
0x83758...880d674
0xb500b...1d03ae8
0x4f947...8f9e6b3
0xb7140...277b79a
0xecd9f...3e3158b
0x769b2...72625e3
0x0a869...eace30f
0x59b22...c12d0cf
0x6a313...3169415
0x5d3be...83c1bf9
0xae28c...5a5f772
0xb58d6...cf1214a
0x86943...2b7fe97
0xd8051...836dca1
0xf102f...6ef5823
0x43c46...3cf661b
0xb2264...f204fe8
0xe9795...5ab67cf
0x11b7b...44775a8
0x9b09d...07cf1fb
0x0bb12...305875a
0x3d4d3...83f35de
0x16eec...73163ca
0xb500c...413328c
0xa6690...5262815
0x251e3...09b4372
0x6edc9...8c58888
0x42909...e858304
0xd9ece...04a737d
0x3677e...0236ea8
0x1ccd3...df73b40
0x45b00...73fcc98
0x23430...1382b62
0x9d4eb...54ed44b
0x3af83...8ed9d3f
0x157e5...0d4fc97
0xf47d4..